In [216]:
import pandas as pd
import re
import json
import ipinfo
import ccy
from forex_python.converter import CurrencyRates

In [217]:
df = pd.read_csv("https://raw.githubusercontent.com/fspot/INFMDI-721/master/lesson5/products.csv", sep=";")

In [218]:
print(df.shape)
df.head()

(200, 5)


,username,ip_address,product,price,infos
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,May contain sugar
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,Contains peanut and fish
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish"


On recupere dans une colonne currency la devise indiquee dans la colonne prix, on retire les currencies de la colonne prix

In [219]:
patterns =  '\D+'
df["currency"] = df["price"].apply(lambda s: re.findall(patterns, s.replace('.', '')))

In [220]:
df["currency"] = df["currency"].apply(lambda l: l[0] if len(l)>0 else '')

In [221]:
df["price"] = df["price"].apply(lambda s: re.sub(pattern2,'',s))

In [84]:
# find country from IP then get currency and convert in euros

In [85]:
token = 'e3be332fea80c9'
handler = ipinfo.getHandler(token)
def geolocatIP(ip):
    try:
        return handler.getDetails(ip).country
    except :
        return "no country"
    

In [87]:
df["country"] = df["ip_address"].apply(lambda s: geolocatIP(s))

In [88]:
len(df[df["country"]=="no country"])

43

In [91]:
df["currency2"] = df["country"].apply(lambda s: ccy.countryccy(s))

In [95]:
# si on avait une ccy donnee qui differe de celle obtenue par l'IP on garde celle donnee
df.loc[df["currency"]!='',"currency2"]= df["currency"]

In [100]:
c = CurrencyRates()
def get_curr_rate(curr):
    try:
        return c.get_rate('EUR',curr)
    except:
        return "no rate"


In [102]:
df["currency_rate"] = df["currency2"].apply(lambda s: get_curr_rate(s))

In [107]:
df.drop("currency", axis=1, inplace=True)

In [112]:
df = df[df["currency_rate"]!="no rate"]
df["price"] = df["price"].apply(lambda c: float(c))

In [113]:
df["price in euros"] = df["price"] / df["currency_rate"]

Supress non ingredient infos from info column

In [188]:

df["infos"] = df["infos"].apply(lambda s: s.replace("Contains",''))

In [189]:
df["infos"] = df["infos"].apply(lambda s: s.replace("Ingredients:",''))

In [190]:
df["infos"] = df["infos"].apply(lambda s: s.replace("May contain",''))

In [191]:
df["infos"] = df["infos"].apply(lambda s: s.replace("and",''))

Split to get a list of the ingredients


In [193]:
df["infos2"] = df["infos"].apply(lambda s: re.split('\s+|[,;.-]\s*', s.strip()))

In [194]:
df.head(5)

,username,ip_address,product,price,infos,infos2
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,sugar,[sugar]
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,peanut fish,"[peanut, fish]"
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,mustard fish,"[mustard, fish]"
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,gluten,[gluten]
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"sugar, egg fish","[sugar, egg, fish]"


In [197]:
pd.concat([df, df["infos2"].str.join('|').str.get_dummies()], axis=1)

,username,ip_address,product,price,infos,infos2,egg,fish,gluten,milk,mustard,peanut,soja,sugar
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,sugar,[sugar],0,0,0,0,0,0,0,1
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,peanut fish,"[peanut, fish]",0,1,0,0,0,1,0,0
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,mustard fish,"[mustard, fish]",0,1,0,0,1,0,0,0
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,gluten,[gluten],0,0,1,0,0,0,0,0
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"sugar, egg fish","[sugar, egg, fish]",1,1,0,0,0,0,0,1
5,bsnozzwell5,226.52.32.70,Oil - Sesame,354.33 MGA,sugar milk,"[sugar, milk]",0,0,0,1,0,0,0,1
6,afairholme6,127.197.204.119,Chicken - Tenderloin,484.83,sugar,[sugar],0,0,0,0,0,0,0,1
7,avowdon7,189.169.17.54,Dc Hikiage Hira Huba,111.56,sugar,[sugar],0,0,0,0,0,0,0,1
8,epridham8,187.129.113.105,Dried Figs,88.05,"sugar, milk fish","[sugar, milk, fish]",0,1,0,1,0,0,0,1
9,tkendrew9,22.32.234.215,Pop - Club Soda Can,861.25,"peanut, sugar, milk fish","[peanut, sugar, milk, fish]",0,1,0,1,0,1,0,1
